In [3]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [4]:
VERSION = 2
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV2"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [5]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
    "SGD": tf.keras.optimizers.legacy.SGD(),
    "SGD_Momentum": tf.keras.optimizers.legacy.SGD(momentum=0.9),
    "RMSprop": tf.keras.optimizers.legacy.RMSprop(),
}

2025-02-28 20:44:22.556627: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-28 20:44:22.556673: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-28 20:44:22.556682: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-28 20:44:22.556918: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 20:44:22.556940: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
LOSS = {"BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()}

In [8]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [9]:
BATCH_SIZE = 64

In [10]:
SUBVERSION = 0
for OPTIMIZER_NAME, _ in OPTIMIZERS.items():
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS[OPTIMIZER_NAME],
        loss=LOSS["BinaryCrossentropy"],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-02-28 20:44:24.982177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.2896 - accuracy: 0.8708 - precision: 0.8945 - recall: 0.9355 - auc: 0.9310

2025-02-28 20:45:19.495595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 69s 243ms/step - loss: 0.2896 - accuracy: 0.8708 - precision: 0.8945 - recall: 0.9355 - auc: 0.9310 - val_loss: 0.2197 - val_accuracy: 0.9148 - val_precision: 0.9212 - val_recall: 0.9528 - val_auc: 0.9705
Epoch 2/10
280/280 [==============================] - 69s 248ms/step - loss: 0.2095 - accuracy: 0.9117 - precision: 0.9346 - recall: 0.9467 - auc: 0.9658 - val_loss: 0.1956 - val_accuracy: 0.9230 - val_precision: 0.9400 - val_recall: 0.9439 - val_auc: 0.9754
Epoch 3/10
280/280 [==============================] - 65s 231ms/step - loss: 0.1925 - accuracy: 0.9197 - precision: 0.9431 - recall: 0.9485 - auc: 0.9710 - val_loss: 0.1902 - val_accuracy: 0.9230 - val_precision: 0.9282 - val_recall: 0.9577 - val_auc: 0.9769
Epoch 4/10
280/280 [==============================] - 61s 217ms/step - loss: 0.1831 - accuracy: 0.9239 - precision: 0.9467 - recall: 0.9506 - auc: 0.9739 - val_loss: 0.1807 - val_accuracy: 0.9281 - val_precision: 0.9476 - val_recall: 

...

67/67 [==============================] - 11s 164ms/step - loss: 0.1631 - accuracy: 0.9337 - precision: 0.9437 - recall: 0.9570 - auc: 0.9819


2025-02-28 21:28:09.277120: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 11s 159ms/step



Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


Epoch 1/10


AttributeError: in user code:

    File "/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/litto/miniconda/envs/TFG/lib/python3.8/site-packages/keras/src/engine/training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)

    AttributeError: 'str' object has no attribute 'minimize'
